In [2]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

# Import Gradio
import gradio as gr

In [3]:
def sms_classification(sms_text_df):
    """
    Perform SMS classification using a pipeline with TF-IDF vectorization and Linear Support Vector Classification.

    Parameters:
    - sms_text_df (pd.DataFrame): DataFrame containing 'text_message' and 'label' columns for SMS classification.

    Returns:
    - text_clf (Pipeline): Fitted pipeline model for SMS classification.

    This function takes a DataFrame with 'text_message' and 'label' columns, splits the data into
    training and testing sets, builds a pipeline with TF-IDF vectorization and Linear Support Vector
    Classification, and fits the model to the training data. 
    The fitted pipeline is returned to make future predictions.
    """
    # Set the features variable to the text message column.
    X = sms_text_df['text_message']  

    # Set the target variable to the "label" column.
    y = sms_text_df['label']

    # Split data into training and testing and set the test_size = 33%
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # Build a pipeline to transform the test set to compare to the training set.
    text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                     ('clf', LinearSVC()),
                    ])

    # Fit the model to the transformed training data and return model.
    return text_clf.fit(X_train, y_train) 

In [ ]:
# Load the dataset into a DataFrame
sms_spam_df = pd.read_csv('./Resources/SMSSpamCollection.csv')
sms_spam_df

FileNotFoundError: [Errno 2] No such file or directory: './Resources/Resources/SMSSpamCollection.csv'

In [ ]:
# Call the sms_classification function with the DataFrame and set the result to the "text_clf" variable
text_clf = sms_classification(sms_spam_df)

In [ ]:
# Create a function called `sms_prediction` that takes in the SMS text and predicts the whether the text is "not spam" or "spam". 
# The function should return the SMS message, and say whether the text is "not spam" or "spam".
def sms_prediction(text):
    """
    Predict the spam/ham classification of a given text message using a pre-trained model.

    Parameters:
    - text (str): The text message to be classified.

    Returns:
    - str: A message indicating whether the text message is classified as spam or not.

    This function takes a text message and a pre-trained pipeline model, then predicts the
    spam/ham classification of the text. The result is a message stating whether the text is
    classified as spam or not.
    """
    # Create a variable that will hold the prediction of a new text.
    text_predict = text_clf.predict([text])

    # Using a conditional if the prediction is "ham" return the message:
    # f'The text message: "{text}", is not spam.' Else, return f'The text message: "{text}", is spam.'
    if text_predict == 'ham':
        return f'The text message: "{text}", is not spam.'
    else:
        return f'The text message: "{text}", is spam.'
    

In [ ]:
# Create a sms_app that takes a textbox for the inputs and has a textbox for the output.  
# Povide labels for each textbox. 
sms_app = gr.Interface(fn=sms_prediction, 
                        inputs=gr.Textbox(label="Enter your text message here"), 
                        outputs=gr.Textbox(label="Prediction"),
                        title="SMS Spam Classification",
                        description="This app classifies SMS messages as spam or not spam using a pre-trained model.")
    
# Launch the app.
sms_app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Test the following text messages. 

---

1. You are a lucky winner of $5000!
2. You won 2 free tickets to the Super Bowl.
3. You won 2 free tickets to the Super Bowl text us to claim your prize.
4. Thanks for registering. Text 4343 to receive free updates on medicare.

In [12]:
#Test the following texts
#You are a lucky winner of $5000!
#You won 2 free tickets to the Super Bowl.
#You won 2 free tickets to the Super Bowl text us to claim your prize.
#Thanks for registering. Text 4343 to receive free updates on medicare.
texts = ["You are a lucky winner of $5000!",
         "You won 2 free tickets to the Super Bowl.",
         "You won 2 free tickets to the Super Bowl text us to claim your prize.",
         "Thanks for registering. Text 4343 to receive free updates on medicare."]
results = [sms_prediction(text) for text in texts]

# Print the results
for text, result in zip(texts, results):
    print(f"Text: {text}\nResult: {result}\n")

Text: You are a lucky winner of $5000!
Result: The text message: "You are a lucky winner of $5000!", is not spam.

Text: You won 2 free tickets to the Super Bowl.
Result: The text message: "You won 2 free tickets to the Super Bowl.", is not spam.

Text: You won 2 free tickets to the Super Bowl text us to claim your prize.
Result: The text message: "You won 2 free tickets to the Super Bowl text us to claim your prize.", is spam.

Text: Thanks for registering. Text 4343 to receive free updates on medicare.
Result: The text message: "Thanks for registering. Text 4343 to receive free updates on medicare.", is spam.

